In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
plt.rcParams["figure.dpi"] = 200
plt.rcParams["font.size"] = 14

import tqdm
import colour
from PIL import Image
from scipy.signal import savgol_filter

from zmqRemoteApi import RemoteAPIClient

from IPython.display import clear_output
import time

from utils import normalize, closedLoop
from elman_opt import ElmanPBRNN, save, load, sigmoid

In [ ]:
%matplotlib inline

trainingPatterns = ['data/circle.npz', 'data/figure8.npz']#, 'data/square.npz', 'data/triangle.npz']
biasArr = np.array([[0.25], [0.75]])#, [.5], [.75]])
seqCutOffFactors = 1.5*np.ones(len(trainingPatterns), dtype=np.int16)
#seqCutOffFactors[1] = 2

dsFactor = 4
initialOffset = 200

xScaleFuncArr = []
yScaleFuncArr = []
xArr = []
yArr = []

#biasArr = []

for i in range(len(trainingPatterns)):

    data = np.load(trainingPatterns[i])
    x, xScale = normalize(data["positions"][initialOffset:-int(len(data["positions"])/seqCutOffFactors[i]):dsFactor,0], returnFunc=True)
    y, yScale = normalize(data["positions"][initialOffset:-int(len(data["positions"])/seqCutOffFactors[i]):dsFactor,1], returnFunc=True)

    #x = x[:len(x)//4]
    #y = y[:len(y)//4]

    xArr.append(x)
    yArr.append(y)
    xScaleFuncArr.append(xScale)
    yScaleFuncArr.append(yScale)
    #biasArr.append(np.zeros(len(trainingPatterns)))
    #biasArr[-1][i] = 1
    
fig, ax = plt.subplots(1, len(trainingPatterns), figsize=(len(trainingPatterns)*4, 4))

for i in range(len(ax)):
    ax[i].plot(xArr[i], yArr[i], '-o')
    ax[i].set_title(f'{trainingPatterns[i]} ({biasArr[i]})')
    ax[i].set_xlabel('$x$')
    ax[i].set_ylabel('$y$')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    
fig.tight_layout()
#plt.savefig('images/example_patterns.png')
plt.show()

In [ ]:
model = ElmanPBRNN(inputDim=2,
                     pbDim=1,
                     contextDim=50,
                     outputDim=2,
                     optimizer='adam',
                     learningRate=1e-3)

numEpochsPerPattern = 10000
forceSymmetrize = False
noiseStrength = .03

In [ ]:
def modelSummary(model, inputArr, trainingBiasArr, targetArr, errorArr, nSteps=150):

    fig, ax = plt.subplots(1, 1 + len(trainingBiasArr), figsize=(4*(1 + len(trainingBiasArr)),5))

    ax[0].plot(errorArr)
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('MSE')
    ax[0].set_yscale('log')
    ax[0].set_title('Learning Curve')

    for i in range(len(trainingBiasArr)):
        #outputArr = model.predict(inputArr[i][0] + np.random.uniform(-.1, .1, size=2), biasArr[i], len(inputArr[i])-1)
        outputArr = np.zeros((nSteps, 2))
        outputArr[0] = np.random.uniform(.1, .9, size=2)
        model.resetContext()
        for j in range(1, len(outputArr)):
            outputArr[j] = model.predict(outputArr[j-1], trainingBiasArr[i], 1, True)[1]

        ax[1+i].set_title(f'Closed Loop Prediction\n$p = {tuple(trainingBiasArr[i])}$')
        ax[1+i].plot(targetArr[i][1:,0], targetArr[i][1:,1], label=r'$\bar y$')
        ax[1+i].plot(outputArr[1:,0], outputArr[1:,1], label=r'$y$')
        ax[1+i].set_xticks([])
        ax[1+i].set_yticks([])
        ax[1+i].legend()

    fig.tight_layout()

    return fig

In [ ]:
if forceSymmetrize:
    inputArr = [np.array(list(zip(xArr[i], yArr[i])))[:-1] for i in range(len(trainingPatterns))] + [np.array(list(zip(xArr[i], yArr[i])))[:-1][::-1] for i in range(len(trainingPatterns))]
    targetArr = [np.array(list(zip(xArr[i], yArr[i])))[1:] for i in range(len(trainingPatterns))] + [np.array(list(zip(xArr[i], yArr[i])))[1:][::-1] for i in range(len(trainingPatterns))]
    errorArr = np.zeros(numEpochsPerPattern * 2 * len(trainingPatterns)) # list to store errors
    
    trainingBiasArr = list(biasArr) + [-b for b in biasArr]
    # Mix up the order in which we train each shape
    trainingOrder = [j for i in range(numEpochsPerPattern) for j in range(2*len(trainingPatterns))]
    np.random.shuffle(trainingOrder)

else:
    inputArr = [np.array(list(zip(xArr[i], yArr[i])))[:-1] for i in range(len(trainingPatterns))]
    targetArr = [np.array(list(zip(xArr[i], yArr[i])))[1:] for i in range(len(trainingPatterns))]
    errorArr = np.zeros(numEpochsPerPattern * len(trainingPatterns)) # list to store errors

    trainingBiasArr = biasArr
    # Mix up the order in which we train each shape
    trainingOrder = [j for i in range(numEpochsPerPattern) for j in range(len(trainingPatterns))]
    np.random.shuffle(trainingOrder)

for epoch in tqdm.tqdm(range(len(trainingOrder))):
    # Context and output
    noise = np.random.normal(0, noiseStrength, size=inputArr[trainingOrder[epoch]].shape)

    contextArr, outputArr = model.forwardSequence(inputArr[trainingOrder[epoch]] + noise, trainingBiasArr[trainingOrder[epoch]])
    errorArr[epoch]  = model.backwardSequence(inputArr[trainingOrder[epoch]] + noise, trainingBiasArr[trainingOrder[epoch]],
                                              contextArr, outputArr, targetArr[trainingOrder[epoch]] + noise)
    model.updateParameters()
    
modelSummary(model, inputArr, trainingBiasArr, targetArr, errorArr)
#plt.savefig('images/learning_curve.png')
plt.show()

In [ ]:
save(model, 'models/scalar_2.npz')

In [ ]:
modelSummary(model, inputArr, trainingBiasArr, targetArr, errorArr, nSteps=400)
#plt.savefig('images/learning_curve_scalar.png')
plt.show()

In [ ]:
untrainedBiasArr = np.array([[.15], [.40], [.60], [.80]])

fig, ax = plt.subplots(1, len(untrainedBiasArr), figsize=(4*len(untrainedBiasArr),5))

for i in range(len(untrainedBiasArr)):
    #outputArr = model.predict(inputArr[i][0] + np.random.uniform(-.1, .1, size=2), biasArr[i], len(inputArr[i])-1)
    outputArr = np.zeros((300, 2))
    outputArr[0] = np.random.uniform(.1, .9, size=2)
    model.resetContext()
    
    for j in range(1, len(outputArr)):
        outputArr[j] = model.predict(outputArr[j-1], untrainedBiasArr[i], 1, True)[1]

    ax[i].set_title(f'Closed Loop Prediction\n$p = {tuple(untrainedBiasArr[i])}$')
    ax[i].plot(outputArr[1:,0], outputArr[1:,1], label=r'$y$')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].legend()

fig.tight_layout()
plt.show()
#modelSummary(model, inputArr, untrainedBiasArr, targetArr, errorArr, nSteps=400)
#plt.show()

In [ ]:
summaryInterval = 3000

if forceSymmetrize:
    inputArr = [np.array(list(zip(xArr[i], yArr[i])))[:-1] for i in range(len(trainingPatterns))] + [np.array(list(zip(xArr[i], yArr[i])))[:-1][::-1] for i in range(len(trainingPatterns))]
    targetArr = [np.array(list(zip(xArr[i], yArr[i])))[1:] for i in range(len(trainingPatterns))] + [np.array(list(zip(xArr[i], yArr[i])))[1:][::-1] for i in range(len(trainingPatterns))]
    errorArr = []
    
    trainingBiasArr = biasArr + [-b for b in biasArr]

else:
    inputArr = [np.array(list(zip(xArr[i], yArr[i])))[:-1] for i in range(len(trainingPatterns))]
    targetArr = [np.array(list(zip(xArr[i], yArr[i])))[1:] for i in range(len(trainingPatterns))]
    errorArr = []

    trainingBiasArr = biasArr

epoch = 0
while True:
    patternIndex = np.random.randint(len(inputArr))
    #print(patternIndex)
    # Context and output
    noise = np.random.normal(0, noiseStrength, size=inputArr[patternIndex].shape)

    contextArr, outputArr = model.forwardSequence(inputArr[patternIndex] + noise, trainingBiasArr[patternIndex])
    error  = model.backwardSequence(inputArr[patternIndex] + noise, trainingBiasArr[patternIndex],
                                    contextArr, outputArr, targetArr[patternIndex] + noise)
    
    errorArr.append(error)
    model.updateParameters()
    epoch += 1
    
    if epoch % summaryInterval == 0:
        fig = modelSummary(model, inputArr, trainingBiasArr, targetArr, errorArr)
        fig.suptitle(f'Epoch: {epoch}')
        fig.tight_layout()
        plt.show()
        save(model, f'models/one_dim_pb_{epoch}.npz')

In [ ]:
trainingPattern

In [ ]:
def genPBTestGif(model, biasArr, filename, predictionStepsPerBias=50, biasSequence=None, startFrames=10, biasStepSmoothness=1, loop=0, fps=10, trailingPoints=5, lateralGridSize=30, colorArr=None, outputFolder='images'):

    testXArr = np.linspace(0, 1, lateralGridSize)
    testYArr = np.linspace(0, 1, lateralGridSize)
    
    if colorArr is None:
        colorArr = colour.Color('Red').range_to(colour.Color('Black'), len(testXArr)*len(testYArr))
        colorArr = [str(c) for c in colorArr]
    
    if biasSequence is None:
        biasSeqArr = np.arange(len(biasArr))
    else:
        biasSeqArr = biasSequence
        
    totalPredictionSteps = predictionStepsPerBias*len(biasSeqArr)
    testBiasArr = np.zeros((totalPredictionSteps, len(biasArr[0])))
    
    for i in range(len(biasSeqArr)):
        testBiasArr[i*predictionStepsPerBias:(i+1)*predictionStepsPerBias,:] = np.repeat([biasArr[biasSeqArr[i]]], predictionStepsPerBias, axis=0)

    if biasStepSmoothness > 0:
        smoothingWindow = int(1+2*np.ceil(biasStepSmoothness*3))
        for i in range(len(testBiasArr[0])):
            testBiasArr[:,i] = savgol_filter(testBiasArr[:,i], 1+6*int(biasStepSmoothness), 1)
            #testBiasArr[testBiasArr[:,i] > 1,i] = 1
            #testBiasArr[testBiasArr[:,i] < 0,i] = 0
            
    #images = [None]*(totalPredictionSteps+startFrames)
    images = []
    
    predictArr = np.zeros((len(testXArr), len(testYArr), totalPredictionSteps+1, 2))
    
    for i in range(len(testXArr)):
        for j in range(len(testYArr)):

            model.resetContext()
            predictArr[i,j,0] = [testXArr[i], testYArr[j]]
            for k in range(1, len(testBiasArr)):
                predictArr[i,j,k] = model.predict(predictArr[i,j,k-1], testBiasArr[k], 1, True)[1]


    fig, ax = plt.subplots(1, 2, figsize=(8,4))
    for i in range(len(biasArr[0])):
        ax[0].plot(testBiasArr[:,i])
        
    ax[0].set_title('Parametric Biases')
    ax[0].set_xlabel('Time step')
    ax[0].axvline(0, linestyle='--', c='r')
    
    for j in range(len(testXArr)):
        for k in range(len(testYArr)):
            ax[1].plot(predictArr[j,k,0,0], predictArr[j,k,0,1], '-o', alpha=.4, c=colorArr[len(testYArr)*j+k])

    fig.tight_layout()
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()

    startImage = Image.frombytes('RGB', canvas.get_width_height(),
                 canvas.tostring_rgb())

    plt.close()

    for i in range(startFrames):#nSteps+1):
        images.append(startImage)
        
    for i in tqdm.tqdm(range(totalPredictionSteps)):#nSteps+1):

        fig, ax = plt.subplots(1, 2, figsize=(8,4))
        
        for l in range(len(testBiasArr[0])):
            ax[0].plot(testBiasArr[:,l])

        ax[0].set_title('Parametric Biases')
        ax[0].set_xlabel('Time step')
        ax[0].axvline(startFrames + i, linestyle='--', c='r')
        
        for j in range(len(testXArr)):
            for k in range(len(testYArr)):
                ax[1].plot(predictArr[j,k,max(i-trailingPoints,0):i,0], predictArr[j,k,max(i-trailingPoints,0):i,1], '-o', alpha=.05, c=colorArr[len(testYArr)*j+k])
                ax[1].plot(predictArr[j,k,i,0], predictArr[j,k,i,1], '-o', alpha=.3, c=colorArr[len(testYArr)*j+k])

        ax[1].set_xlim([0,1])
        ax[1].set_ylim([0,1])

        fig.tight_layout()
        canvas = plt.get_current_fig_manager().canvas
        canvas.draw()

        images.append(Image.frombytes('RGB', canvas.get_width_height(),
                     canvas.tostring_rgb()))
        #plt.show()
        plt.close()

    images[0].save(f'{outputFolder}/{filename}', save_all=True, append_images=images[1:], duration=fps, loop=loop)


In [ ]:
genPBTestGif(model, trainingBiasArr, 'pbrnn_4_pattern_one_dim_untrained_scan_asymm.gif', predictionStepsPerBias=100,
             startFrames=1, biasStepSmoothness=10., biasSequence=None)

In [ ]:
negBiasArr = [-b for b in biasArr]

genPBTestGif(model, [biasArr[0], negBiasArr[0], biasArr[0]], 'pbrnn_test_6.gif', predictionStepsPerBias=60,
             startFrames=1, biasStepSmoothness=.1, biasSequence=[0, 1, 0])

In [ ]:
customBiasArr = [np.array([1, 0, 0, 0]), np.array([1., 1., 0., 0.]), np.array([0., 1., 1., 0.]), np.array([.5, .5, .5, .5])]

genPBTestGif(model, customBiasArr, 'pbrnn_4_pattern_untrained_4.gif', predictionStepsPerBias=60,
             startFrames=1, biasStepSmoothness=0, biasSequence=[0, 1, 2, 3])

In [ ]:
customBiasArr = [np.array([1, 0, 0, 0]), np.array([0, 1., 1., 0])]

genPBTestGif(model, customBiasArr, 'pbrnn_4_pattern_untrained_2.gif', predictionStepsPerBias=60,
             startFrames=1, biasStepSmoothness=1., biasSequence=[0, 1, 1, 0])

## Training Debugging

In [ ]:
%matplotlib inline

epochArr = np.array([1000, 5000, 10000, 15000, 25000])
epochDiffs = np.append(epochArr[0], epochArr[1:] - epochArr[:-1])

inputArr = np.array(list(zip(xArr, yArr)))[:-1]
targetArr = np.array(list(zip(xArr, yArr)))[1:]

model = ElmanNetwork(inputDim=2,
                     contextDim=20,
                     outputDim=2,
                     optimizer='adam')

allOutputArr = []
errorArr = []

for numEpochs in epochDiffs:
    
    for epoch in tqdm.tqdm(range(numEpochs)):
        # Context and output
        contextArr, outputArr = model.forwardSequence(inputArr)
        err  = model.backwardSequence(inputArr, contextArr, outputArr, targetArr)
        errorArr.append(err)
        model.updateParameters()
        
    outputArr = model.predict(inputArr[0,0] + np.random.uniform(-.1, .1, size=2), len(inputArr)-1)

    allOutputArr.append(outputArr)
    
fig, ax = plt.subplots(1, len(allOutputArr), figsize=(len(allOutputArr)*3+5,4))

ax[0].plot(errorArr)
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('MSE')
ax[0].set_yscale('log')
ax[0].set_title('Learning Curve')

for i in range(len(allOutputArr)):
    ax[i].set_title('Closed Loop Prediction')
    ax[i].plot(targetArr[1:,0], targetArr[1:,1], label=r'$\bar y$')
    ax[i].plot(allOutputArr[i][1:,0], allOutputArr[i][1:,1], label=r'$y$')
    ax[i].legend()
    ax[i].set_title(f'{epochArr[i]} Epochs')
    ax[0].axvline(epochArr[i], c='r', linestyle='--')

fig.tight_layout()

#plt.savefig('images/circle_training.png')
plt.show()

In [ ]:
#open connection to CoppeliaSim, must be running
client = RemoteAPIClient()
sim = client.getObject('sim')

#open scene in the simulator:
import os 
dir_path = os.getcwd()

sim.loadScene(dir_path+'/torobo.ttt')

#we will use synchronous mode, client.step() will perform one simulation step
#otherwise the simulator runs freely, e.g. time can pass between mutliple simulator calls:
#e.g. sending commands to multiple joints may happen at different times
client.setStepping(True)

In [ ]:
#Read object ids from scene
hands = [sim.getObject('./tip_left'), sim.getObject('./tip_right')]
rightShoulderJoints = [[sim.getObject(f'/world_visual[{i}]/right_arm_joint_1'), sim.getObject(f'/world_visual[{i}]/right_arm_joint_2')] for i in range(2)]

visionSensors = [sim.getObject('/world_visual[0]/kinect_rgb'), sim.getObject('/world_visual[1]/kinect_rgb')]
trackerSphere = sim.getObject('/world_visual[1]/tracker_ball')

def trackRedBall(img, redTol=30000, normalize=True):
    redPixels = np.where(np.sum((img - np.array([255, 0, 0]))**2, axis=-1) < redTol)
    avg = np.mean(redPixels, axis=-1)
    if normalize:
        avg = avg / img.shape[:2]
        
    trackImg = np.zeros_like(img)
    trackImg[redPixels] = [255, 0, 0]
    
    return avg, trackImg

In [ ]:
objects = sim.getObjectsInTree(sim.getObject('/world_visual[1]'))
for o in objects:
    print(sim.getObjectAlias(o))

In [ ]:
sim.startSimulation()
nSteps = 250
imgArr = [[], []]
ballPosArr = []
trackImgArr = []
jointPosArr = []

biasSeqArr = [0, 1]
biasStepSmoothness = 1.
testBiasArr = np.zeros((nSteps, len(biasArr[0])))
predictionStepsPerBias = nSteps // len(biasSeqArr)

for i in range(len(biasSeqArr)):
    testBiasArr[i*predictionStepsPerBias:(i+1)*predictionStepsPerBias,:] = np.repeat([biasArr[biasSeqArr[i]]], predictionStepsPerBias, axis=0)

if biasStepSmoothness > 0:
    smoothingWindow = int(1+2*np.ceil(biasStepSmoothness*3))
    for i in range(len(testBiasArr[0])):
        testBiasArr[:,i] = savgol_filter(testBiasArr[:,i], 1+6*int(biasStepSmoothness), 1)

imitationTrackingWindowSize = 30

try:
    # The wrists are bent to start, so let's straighten them out
    sim.setJointTargetPosition(sim.getObject('/world_visual[0]/right_arm_joint_6'), 0)
    sim.setJointTargetPosition(sim.getObject('/world_visual[0]/left_arm_joint_6'), 0)
    sim.setJointTargetPosition(sim.getObject('/world_visual[0]/right_arm_joint_1'), 0)
    sim.setJointTargetPosition(sim.getObject('/world_visual[0]/right_arm_joint_2'), 0)

    # Set the ball attached to the second robots hand to be red, so the
    # first one can track it
    #sim.setShapeColor(trackerSphere, None, sim.colorcomponent_ambient_diffuse, [1, 0, 0])

    handPos = np.zeros((2,nSteps,3))
    tArr = np.zeros(nSteps)

    jointPosArr.append(np.array(inputArr[0][0,0]))

    for i in tqdm.tqdm(range(nSteps)):

        jointPos = model.predict(jointPosArr[-1], testBiasArr[i], 1, True)[1]
        jointPosArr.append(jointPos)
        
        sim.setJointTargetPosition(rightShoulderJoints[1][0], xScale(jointPos[0]))
        sim.setJointTargetPosition(rightShoulderJoints[1][1], yScale(jointPos[1]))

        # Read the positions of the hands
        for j in range(2):
            handPos[j,i] = sim.getObjectPosition(hands[j], sim.handle_world)

        # See if the other robot can identify the bias vector
        if len(ballPosArr) > imitationTrackingWindowSize:
            trackingData = ballPositionArr[-imitationTrackingWindowSize:][::-1]
            backpropInputData = trackingData[:-1]
            backpropTargetData = trackingData[1:]
            
        #next time step
        client.step()
        tArr[i] = sim.getSimulationTime()
        
        for j in range(len(visionSensors)):
            img, imgSize = sim.getVisionSensorImg(visionSensors[j])
            img = np.reshape(np.frombuffer(img, np.uint8), (*imgSize[::-1], 3))[::-1]
            imgArr[j].append(img)
            
        # Track the ball in the second robot's hand
        pos, trackImg = trackRedBall(imgArr[0][-1], normalize=False)
        ballPosArr.append(pos)
        trackImgArr.append(trackImg)

finally:
    sim.stopSimulation()

In [ ]:
for i in range(nSteps):

    clear_output(wait=True)
    fig, ax = plt.subplots(1, 2, figsize=(9,4))

    ax[0].imshow(imgArr[0][i])

    avg, trackImg = trackRedBall(imgArr[0][i])
    ax[1].imshow(trackImgArr[i])

    fig.tight_layout()
    plt.show()
    
    time.sleep(.5)

In [ ]:
pilImgArr = []

for i in range(20, len(imgArr[0])):
    
    fig, ax = plt.subplots(1, 1, figsize=(12.8/1.2, 9.6/1.2))
    ax.imshow(imgArr[1][i])
    ax.set_xticks([])
    ax.set_yticks([])
    
    inset = inset_axes(ax, width="30%", height="25%", loc=3)
    for j in range(len(testBiasArr[0])):
        inset.plot(testBiasArr[:,j])
    inset.set_xticks([])
    inset.set_yticks([])
    inset.set_xlabel('Time')
    inset.axvline(i, linestyle='--', c='r')
    
    #fig.tight_layout()
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()

    pilImgArr.append(Image.frombytes('RGB', canvas.get_width_height(),
                 canvas.tostring_rgb()))

    plt.close()
    
pilImgArr[0].save(f'images/basic_robot_example.gif', save_all=True, append_images=pilImgArr[1:], duration=10, loop=0)

In [ ]:
pilImgArr = []
posArr = np.array(ballPosArr)

for i in range(20, len(imgArr[0])):
    
    fig, ax = plt.subplots(1, 1, figsize=(12.8 / 1.2, 9.6 / 1.2))
    ax.imshow(imgArr[0][i])
    ax.set_xticks([])
    ax.set_yticks([])
    
    inset = inset_axes(ax, width="30%", height="25%", loc=3)
    for j in range(len(testBiasArr[0])):
        inset.plot(testBiasArr[:,j])

    inset.axvline(i, linestyle='--', c='r')
    
    inset2 = inset_axes(ax, width="30%", height="25%", loc=4)
    inset2.imshow(trackImgArr[i])
    inset2.plot(posArr[20:i,1], posArr[20:i,0], alpha=.4)
    #inset2.set_xticks([])
    #inset2.set_yticks([])

    #fig.tight_layout()
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()

    pilImgArr.append(Image.frombytes('RGB', canvas.get_width_height(),
                 canvas.tostring_rgb()))

    plt.close()
    
pilImgArr[0].save(f'images/robot_ball_tracking.gif', save_all=True, append_images=pilImgArr[1:], duration=10, loop=0)

In [ ]:
plt.plot(np.array(ballPosArr)[:,0], np.array(ballPosArr)[:,1])
plt.show()

In [ ]:
#%matplotlib qt

fig = plt.figure()

ax = fig.add_subplot(1, 2, 1, projection='3d')

#ax.scatter(handPos[0,:,0], handPos[0,:,1], handPos[0,:,2], label='Left')
ax.scatter(handPos[1,:,0], handPos[1,:,1], handPos[1,:,2], label='Right')
ax.set_title('Hand Positions')
#ax.set_xlabel('Time [s]')?
ax.legend()

ax2 = fig.add_subplot(1, 2, 2)

ax2.plot(predictArr[:,0], predictArr[:,1])
ax2.set_xlabel('Joint 1 angle')
ax2.set_ylabel('Joint 2 angle')

fig.tight_layout()
plt.show()

In [ ]:
sim.stopSimulation()